In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#reading data
movies_df = pd.read_csv('../../Dataset/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('../../Dataset/ml-latest-small/ratings.csv')

In [50]:
#making new df for calculating average rating without unnecessary features
df_for_avg_rat = ratings_df.drop(columns=['userId', 'timestamp'])
df_for_avg_rat.head()
#sorted by movieId
df_for_avg_rat.sort_values(by='movieId', inplace=True)
df_for_avg_rat
#creating average and total rating dictionaries
avr_rat = {}
tot_rat = {}
temp = 0
counter = 0
for i in range(len(df_for_avg_rat)):
    if df_for_avg_rat['movieId'].iloc[i] == df_for_avg_rat['movieId'].iloc[-1] or df_for_avg_rat['movieId'].iloc[i] == df_for_avg_rat['movieId'].iloc[i+1]:
        temp += df_for_avg_rat['rating'].iloc[i]
        counter+=1
    elif df_for_avg_rat['movieId'].iloc[i] != df_for_avg_rat['movieId'].iloc[i+1]:
        temp += df_for_avg_rat['rating'].iloc[i]
        avr_rat[df_for_avg_rat['movieId'].iloc[i]] = temp/counter
        tot_rat[df_for_avg_rat['movieId'].iloc[i]] = counter
        temp = 0
        counter = 0
    else:
        break

#tot_rat dictionary converted to DF
tot_rat_df = pd.DataFrame.from_dict(tot_rat, orient='index')
tot_rat_df.columns = ['totalRating']
tot_rat_df['movieId'] = tot_rat_df.index
tot_rat_df = tot_rat_df.reindex(columns=['movieId', 'totalRating'])
tot_rat_df

#avr_rat dictionary converted to DF
avr_rat_df = pd.DataFrame.from_dict(avr_rat, orient='index')
avr_rat_df.columns = ['averageRating']
avr_rat_df['movieId'] = avr_rat_df.index
avr_rat_df = avr_rat_df.reindex(columns=['movieId', 'averageRating'])
avr_rat_df

#deleting the movies that have less than 20 ratings
filtered_trd = tot_rat_df[tot_rat_df['totalRating'] >= 20]
filtered_trd.sort_values(by=['totalRating'])

#merged both DF's
ratingsWoUnusedDF = pd.merge(avr_rat_df, filtered_trd, on='movieId')
ratingsWoUnusedDF.sort_values(by=['movieId'])

C:\Users\burak\AppData\Local\Temp\ipykernel_10112\3803296616.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  avr_rat[df_for_avg_rat['movieId'].iloc[i]] = temp/counter


,movieId,averageRating,totalRating
0,1,3.939252,214
1,2,3.463303,109
2,3,3.323529,51
3,5,3.135417,48
4,6,3.985149,101
...,...,...,...
1230,148626,4.120000,25
1231,152081,4.016129,31
1232,164179,4.140000,25
1233,166528,4.076923,26


CREATING USER TRAINING DATA

In [51]:
#ratings data (is going to be user data later)
ratings_df.head()
#drop timestamp column
ratings_df = ratings_df.drop(columns=['timestamp'])

In [52]:
filtered_movieids = ratingsWoUnusedDF['movieId']
filtered_movieids

0            1
1            2
2            3
3            5
4            6
         ...  
1230    148626
1231    152081
1232    164179
1233    166528
1234    168252
Name: movieId, Length: 1235, dtype: int64

In [53]:
ratings_df = pd.merge(ratings_df, filtered_movieids, on='movieId')
ratings_df

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
...,...,...,...
66653,572,2013,1.0
66654,577,2013,3.0
66655,590,2013,2.5
66656,597,2013,3.0


In [54]:
movies_df = pd.merge(movies_df, filtered_movieids, on='movieId')

movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
1230,148626,"Big Short, The (2015)",Drama
1231,152081,Zootopia (2016),Action|Adventure|Animation|Children|Comedy
1232,164179,Arrival (2016),Sci-Fi
1233,166528,Rogue One: A Star Wars Story (2016),Action|Adventure|Fantasy|Sci-Fi


In [56]:
movies_df.to_csv('./csv/movie_list.csv', index=False)

In [11]:
genres = movies_df['genres'].str.get_dummies('|')
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1231,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1232,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1233,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [12]:
#merging movieid's with their genres
movies_with_genres = pd.concat([movies_df,genres], axis=1)
movies_with_genres

,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,148626,"Big Short, The (2015)",Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1231,152081,Zootopia (2016),Action|Adventure|Animation|Children|Comedy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1232,164179,Arrival (2016),Sci-Fi,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1233,166528,Rogue One: A Star Wars Story (2016),Action|Adventure|Fantasy|Sci-Fi,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [14]:
user_df = pd.merge(ratings_df, movies_with_genres, left_on='movieId', right_on='movieId')
user_df.drop(columns=['title','genres'], axis=1, inplace=True)
user_df.sort_values(by=['userId'])

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10766,1,2046,4.0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10790,1,2054,4.0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
10858,1,2058,5.0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
10888,1,2078,5.0,0,0,1,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34501,610,5989,3.5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
39792,610,1358,4.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17949,610,2288,5.0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0
36555,610,45517,3.0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#how many ratings each user gave as a dataframe
rating_count = user_df.pivot_table(index=['userId'], aggfunc='size')
rating_count = pd.DataFrame(rating_count)
rating_count.columns = ['rating count']
rating_count['userId'] = rating_count.index
rating_count.index.name = None
rating_count

,rating count,userId
1,180,1
2,23,2
3,22,3
4,136,4
5,41,5
...,...,...
606,454,606
607,140,607
608,581,608
609,25,609


In [19]:
#merge rating count with user_df
user_df = pd.merge(user_df, rating_count, left_on='userId', right_on='userId')
user_df.sort_values(by=['userId'])

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating count
0,1,1,4.0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,180
116,1,2273,4.0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,180
117,1,2291,5.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,180
118,1,2329,5.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,180
119,1,2353,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40079,610,107406,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,518
40080,610,4105,5.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,518
40081,610,7022,5.0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,518
40072,610,54995,3.5,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,518


In [20]:
#getting column names for the for loop below
columnnames = user_df.columns
columnnames

Index(['userId', 'movieId', 'rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'rating count'],
      dtype='object')

In [21]:
#multiplying every rating by genres to find average rating each user gave for each genre
for i in range(3,22):
    user_df[columnnames[i]] = user_df[columnnames[i]] * user_df['rating']
user_df.sort_values('userId', inplace=True)
user_df


,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating count
0,1,1,4.0,0.0,4.0,4.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180
116,1,2273,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,180
117,1,2291,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,180
118,1,2329,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180
119,1,2353,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40079,610,107406,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,518
40080,610,4105,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,518
40081,610,7022,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,518
40072,610,54995,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,518


In [22]:
#replacing zero values with nan because we don't want zero value counts to affect average
user_df.replace(0, np.nan, inplace=True)
#grouping by user id so it stays the same
grouped = user_df.groupby(['userId'])

def sum_and_divide(x):
    return x.sum() / x.notnull().sum()

user_data_with_average_ratings = grouped.apply(sum_and_divide)
user_data_with_average_ratings

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating count
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,1769.555556,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,NaN,...,3.300000,NaN,4.777778,4.400000,4.277778,4.243243,4.276596,4.500000,4.285714,180.0
2,2.0,63473.652174,3.934783,3.954545,4.166667,NaN,NaN,4.200000,4.000000,NaN,...,3.000000,3.750000,NaN,4.000000,4.500000,3.875000,3.888889,4.500000,3.500000,23.0
3,3.0,4863.909091,1.568182,3.125000,3.357143,0.500000,0.500000,0.500000,0.500000,NaN,...,4.000000,NaN,0.500000,NaN,0.500000,3.250000,3.625000,0.500000,NaN,22.0
4,4.0,1779.088235,3.544118,3.190476,3.760000,4.000000,4.111111,3.522388,3.863636,NaN,...,4.333333,3.000000,4.076923,3.000000,3.378378,2.727273,3.250000,3.000000,3.250000,136.0
5,5.0,343.243902,3.609756,3.111111,3.250000,4.333333,4.111111,3.428571,3.727273,NaN,...,3.000000,3.666667,4.400000,4.000000,3.000000,2.500000,3.555556,3.333333,3.000000,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,606.0,7757.927313,3.666300,3.298165,3.550459,3.800000,3.560606,3.505848,3.816176,4.000000,...,3.538462,3.115385,3.794118,3.845238,3.736000,3.570312,3.637255,3.854167,3.375000,454.0
607,607.0,1806.771429,3.742857,3.688525,3.435897,3.000000,3.363636,3.333333,3.769231,NaN,...,4.136364,5.000000,3.250000,4.600000,3.541667,3.303030,4.021739,4.000000,4.000000,140.0
608,608.0,3904.986231,3.271945,3.447115,3.293548,3.179487,2.640625,2.936214,3.643519,3.333333,...,3.568966,4.000000,2.833333,3.683673,2.922619,3.402985,3.623596,3.593750,2.666667,581.0


In [23]:
#rewinding the zero values
user_data_with_average_ratings.replace(np.nan, 0, inplace=True)
#dropping movieid column because we don't need it
user_data_with_average_ratings.drop(columns='movieId', axis=1, inplace=True)
user_data_with_average_ratings

,userId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating count
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.000000,4.480769,...,3.300000,0.000000,4.777778,4.400000,4.277778,4.243243,4.276596,4.500000,4.285714,180.0
2,2.0,3.934783,3.954545,4.166667,0.000000,0.000000,4.200000,4.000000,0.000000,3.933333,...,3.000000,3.750000,0.000000,4.000000,4.500000,3.875000,3.888889,4.500000,3.500000,23.0
3,3.0,1.568182,3.125000,3.357143,0.500000,0.500000,0.500000,0.500000,0.000000,0.500000,...,4.000000,0.000000,0.500000,0.000000,0.500000,3.250000,3.625000,0.500000,0.000000,22.0
4,4.0,3.544118,3.190476,3.760000,4.000000,4.111111,3.522388,3.863636,0.000000,3.424242,...,4.333333,3.000000,4.076923,3.000000,3.378378,2.727273,3.250000,3.000000,3.250000,136.0
5,5.0,3.609756,3.111111,3.250000,4.333333,4.111111,3.428571,3.727273,0.000000,3.772727,...,3.000000,3.666667,4.400000,4.000000,3.000000,2.500000,3.555556,3.333333,3.000000,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,606.0,3.666300,3.298165,3.550459,3.800000,3.560606,3.505848,3.816176,4.000000,3.828571,...,3.538462,3.115385,3.794118,3.845238,3.736000,3.570312,3.637255,3.854167,3.375000,454.0
607,607.0,3.742857,3.688525,3.435897,3.000000,3.363636,3.333333,3.769231,0.000000,3.981818,...,4.136364,5.000000,3.250000,4.600000,3.541667,3.303030,4.021739,4.000000,4.000000,140.0
608,608.0,3.271945,3.447115,3.293548,3.179487,2.640625,2.936214,3.643519,3.333333,3.405128,...,3.568966,4.000000,2.833333,3.683673,2.922619,3.402985,3.623596,3.593750,2.666667,581.0


In [24]:
#get rating count a list for repeating process
rating_count_as_list = user_data_with_average_ratings["rating count"].values.tolist()
rating_count_as_list

[180.0,
 23.0,
 22.0,
 136.0,
 41.0,
 162.0,
 123.0,
 47.0,
 22.0,
 78.0,
 54.0,
 29.0,
 25.0,
 42.0,
 121.0,
 87.0,
 98.0,
 366.0,
 394.0,
 162.0,
 283.0,
 105.0,
 77.0,
 100.0,
 19.0,
 21.0,
 99.0,
 331.0,
 63.0,
 33.0,
 43.0,
 82.0,
 102.0,
 71.0,
 22.0,
 42.0,
 21.0,
 77.0,
 90.0,
 79.0,
 178.0,
 290.0,
 87.0,
 33.0,
 322.0,
 42.0,
 86.0,
 32.0,
 20.0,
 146.0,
 214.0,
 99.0,
 14.0,
 29.0,
 24.0,
 44.0,
 340.0,
 103.0,
 83.0,
 21.0,
 37.0,
 246.0,
 245.0,
 448.0,
 31.0,
 238.0,
 32.0,
 852.0,
 40.0,
 51.0,
 31.0,
 41.0,
 142.0,
 81.0,
 60.0,
 103.0,
 29.0,
 53.0,
 48.0,
 118.0,
 26.0,
 168.0,
 93.0,
 183.0,
 13.0,
 63.0,
 18.0,
 49.0,
 131.0,
 22.0,
 403.0,
 15.0,
 82.0,
 56.0,
 123.0,
 69.0,
 31.0,
 64.0,
 42.0,
 117.0,
 44.0,
 54.0,
 292.0,
 173.0,
 368.0,
 30.0,
 33.0,
 39.0,
 96.0,
 37.0,
 292.0,
 65.0,
 74.0,
 28.0,
 96.0,
 85.0,
 147.0,
 20.0,
 150.0,
 19.0,
 58.0,
 232.0,
 45.0,
 46.0,
 154.0,
 38.0,
 20.0,
 23.0,
 130.0,
 28.0,
 60.0,
 287.0,
 35.0,
 35.0,
 214.0,
 86.0,
 11

In [25]:
#repeat every column by userid according to the the users rating counts
repeated_user_df = user_data_with_average_ratings.loc[user_data_with_average_ratings.index.repeat(rating_count_as_list)]
repeated_user_df.rename(columns={'rating': 'average rating'}, inplace=True)
repeated_user_df


,userId,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating count
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,4.480769,...,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714,180.0
1,1.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,4.480769,...,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714,180.0
1,1.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,4.480769,...,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714,180.0
1,1.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,4.480769,...,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714,180.0
1,1.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,4.480769,...,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,610.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,4.058673,...,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000,518.0
610,610.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,4.058673,...,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000,518.0
610,610.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,4.058673,...,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000,518.0


In [26]:
#reposition the columns beacuse it is easier to read
new_column_order = ['userId', 'rating count', 'average rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western']
repeated_user_df = repeated_user_df.reindex(columns=new_column_order)
repeated_user_df

,userId,rating count,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,610.0,518.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,...,4.833333,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000
610,610.0,518.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,...,4.833333,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000
610,610.0,518.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,...,4.833333,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000


In [28]:
#creating x user training data (dropping userid, ratingcount, average rating)
x_user_train = repeated_user_df
x_user_train

,userId,rating count,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
1,1.0,180.0,4.422222,4.328947,4.455882,4.764706,4.760000,4.328571,4.361111,0.0,...,5.000000,3.300000,0.000000,4.777778,4.40,4.277778,4.243243,4.276596,4.500000,4.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,610.0,518.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,...,4.833333,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000
610,610.0,518.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,...,4.833333,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000
610,610.0,518.0,3.957529,3.897778,3.930303,3.985714,3.893939,3.868280,4.042857,4.0,...,4.833333,3.944444,3.916667,3.785714,4.05,3.842857,3.958333,3.969101,4.071429,4.300000


CREATING USER TEST DATA

In [29]:
#y train should contain movie ratings
y_train_user = user_df['rating']
y_train_user

0        4.0
116      4.0
117      5.0
118      5.0
119      5.0
        ... 
40079    5.0
40080    5.0
40081    5.0
40072    3.5
39847    5.0
Name: rating, Length: 66658, dtype: float64

CREATING ITEM TRAINING DATA

In [30]:
#checking ratings data. we want it to be like movieId | year | average rating | (genres)
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
1230,148626,"Big Short, The (2015)",Drama
1231,152081,Zootopia (2016),Action|Adventure|Animation|Children|Comedy
1232,164179,Arrival (2016),Sci-Fi
1233,166528,Rogue One: A Star Wars Story (2016),Action|Adventure|Fantasy|Sci-Fi


In [31]:
#subtracting the year from the movie title 
pattern = '\((\d{4})\)'
movies_df['year'] = movies_df.title.str.extract(pattern, expand=False)
movies_df.drop(columns=['title','genres'], axis=1, inplace=True)

Burada bazı filmlerin başlığında film yılı yazmadığı için NaN değer alan year değerleri vardı, filtreleme yapınca kalmadı

In [32]:
pd.isnull(movies_df['year']).sort_values()

0       False
827     False
826     False
825     False
824     False
        ...  
408     False
407     False
406     False
413     False
1234    False
Name: year, Length: 1235, dtype: bool

In [34]:

ratings_df.sort_values(by=['rating'])

,userId,movieId,rating
58353,298,2724,0.5
41252,489,784,0.5
17906,21,2288,0.5
38344,517,40629,0.5
57480,489,4299,0.5
...,...,...,...
18336,305,32,5.0
52689,389,832,5.0
52692,456,832,5.0
18455,602,45,5.0


In [35]:
#finding how many unique movieId's
ratings_df['movieId']. nunique()

1235

In [36]:
#finding average rating for each movie
grouped_movies = user_df.groupby(['movieId'])

movie_ave_rating = grouped_movies.apply(sum_and_divide)
movie_ave_rating.drop(columns=['userId', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'rating count'], axis= 1, inplace=True)
movie_ave_rating.rename(columns={"movieId": "movieId", "rating": "average rating"}, inplace=True)
movie_ave_rating.index.name = None
movie_ave_rating

,movieId,average rating
1,1.0,3.920930
2,2.0,3.431818
3,3.0,3.259615
5,5.0,3.071429
6,6.0,3.946078
...,...,...
148626,148626.0,3.961538
152081,152081.0,3.890625
164179,164179.0,3.980769
166528,166528.0,3.925926


In [37]:
#merge average rating with movies_df
movies_df = pd.merge(movies_df, movie_ave_rating)
#x_item_train = pd.merge(movies_df[['movieId', 'year']], movie_ave_rating[['average rating']], on='movieId')
movies_df

,movieId,year,average rating
0,1,1995,3.920930
1,2,1995,3.431818
2,3,1995,3.259615
3,5,1995,3.071429
4,6,1995,3.946078
...,...,...,...
1230,148626,2015,3.961538
1231,152081,2016,3.890625
1232,164179,2016,3.980769
1233,166528,2016,3.925926


In [38]:
movies_with_genres


,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,148626,"Big Short, The (2015)",Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1231,152081,Zootopia (2016),Action|Adventure|Animation|Children|Comedy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1232,164179,Arrival (2016),Sci-Fi,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1233,166528,Rogue One: A Star Wars Story (2016),Action|Adventure|Fantasy|Sci-Fi,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [41]:
#creating not repeated item dataframe
x_item_train_not_repeated = pd.merge(movies_df,movies_with_genres)
x_item_train_not_repeated.drop(columns=['title', 'genres'], axis= 1, inplace=True)
x_item_train_not_repeated

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,3.431818,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,3.259615,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,5,1995,3.071429,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,1995,3.946078,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,148626,2015,3.961538,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1231,152081,2016,3.890625,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1232,164179,2016,3.980769,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1233,166528,2016,3.925926,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [42]:
user_df

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating count
0,1,1,4.0,NaN,4.0,4.0,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180
116,1,2273,4.0,4.0,NaN,NaN,NaN,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,180
117,1,2291,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,180
118,1,2329,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180
119,1,2353,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40079,610,107406,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,518
40080,610,4105,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,518
40081,610,7022,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,518
40072,610,54995,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,518


In [43]:
#we need to merge according to movieid order in user_df. using locate to get the order of the movieids
y_train_sort = user_df.loc[:, ['movieId', 'rating']]
y_train_sort

,movieId,rating
0,1,4.0
116,2273,4.0
117,2291,5.0
118,2329,5.0
119,2353,5.0
...,...,...
40079,107406,5.0
40080,4105,5.0
40081,7022,5.0
40072,54995,3.5


In [44]:
#merging
x_item_train_repeated = pd.merge(y_train_sort, x_item_train_not_repeated, on='movieId')
x_item_train_repeated.drop(columns=['rating'],inplace=True)
x_item_train_repeated

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66653,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66654,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66655,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66656,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
#dropping moiveid because it's not gonna be needed
x_train_item = x_item_train_repeated
x_train_item

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1995,3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66653,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66654,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66655,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66656,2013,1972,3.095238,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
"""x_train_item.drop(columns=['year'],inplace=True)
x_train_item"""

,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,3.92093,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,3.92093,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3.92093,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3.92093,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,3.92093,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,4.00000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
100832,3.00000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
100833,4.00000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
100834,3.50000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0


In [46]:
x_train_item.to_csv('./csv/x_train_item.csv', index=False, header=False)
x_user_train.to_csv('./csv/x_train_user.csv', index=False, header=False)
y_train_user.to_csv('./csv/y_train.csv', index=False, header=False)

In [48]:
x_item_train_not_repeated.to_csv('./csv/item_vecs.csv', index=False, header=False)